In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Maximum Recommended Age'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=702
attribute_id_input=5097

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]   

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    tuples=tuple(split_digits(s, *args, **kwargs))
    print(tuples)
    return tuples 

In [6]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
len(df)

5226

In [33]:
# # \b\sage(?!.{0,30}and.?up)|(year(?!.?.{0,30}warranty)|(up to)
# regex_pattern= r"(?i)(30.?month(?!.?.{0,30}and up))|()"
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# print(len(df[df['matches'].astype(str)!='[]']))
# df[df['matches'].astype(str)!='[]'][0:500]

In [42]:
# \b\sage(?!.{0,30}and.?up)|(year(?!.?.{0,30}warranty)|(up to)|(30.?month(?!.?.{0,30}and up))
regex_pattern= r"(?i)(11.?year(?!.?.{0,30}warranty)(?!.?.{0,30}and up))|()"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[df['matches'].astype(str)!='[]']))
df[df['matches'].astype(str)!='[]'][0:500]

1


product_id external_id                     product_name  \
231   988654366    65928202  Gener8 130-Piece Deluxe Art Set   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  long_desc  \
231  Have your little artist develop their creative minds with the 130-Piece Deluxe Art Set. This colorful set includes a variety of crayons, oil pastels, markers, color pencils, watercolor options, and more. WARNING: Choking Hazard Small Parts. Not for children under 3 yrs. Suitable for ages 6-11 years 130-piece set includes: 24 crayons 24 oil pastels 36 markers 24 color pencils 12 watercolor cakes 2 glue tubes Paintbrush Palette Pencil Eraser Sharpener Scissors Staples Ruler Case Art supplies are enclosed in a wooden case with claps Measures 15\" L x 2\" W x 12.25\" H Assembly required Imported   

     customer_id attributes                     buckets  bucket_id values  \
231            5       None  Artistic Crafting Supplies       9359   None   

       matches  
231  [11 year]

In [7]:
df

product_id external_id  \
0      817403739    69427978   
1     1285951871    69717698   
2     1180598556    66656920   
3      988477835    69597629   
4      988462350    69532918   
...          ...         ...   
5221   817170570    45863684   
5222   817462857    45102950   
5223   817168355    45102806   
5224   817462838    45102738   
5225  1206945865    69708848   

                                                           product_name  \
0                                    Sassy® Whirling Waterfall Bath Toy   
1                                         Magic Years® 3-Piece Duck Set   
2                             Living Textiles Theo Elephant Knit Rattle   
3                     Hape Swimmer Teddy Wind-Up Bath Toy in Brown/Blue   
4                Capelli New York 5-Piece Farm Animal Finger Puppet Set   
...                                                                 ...   
5221                                         Rubik's® 5 x 5 Brainteaser   
5222                                   Hanayama Level 6 Elk Cast Puzzle   
5223                                Hanayama Level 4 Marble Cast Puzzle   
5224                                Hanayama Level 3 Violin Cast Puzzle   
5225  BRITAX® B-Safe® Gen2™ Infant Car Seat in Eclipse Black (SafeWash)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [7]:
df[curation_col]='n/a'
df_na=df[['external_id',curation_col]]
df_na

external_id Q:care
0       44040239    n/a
1       69595486    n/a
2       41546154    n/a
3       20722982    n/a
4       20723132    n/a
...          ...    ...
4360    47552012    n/a
4361    20596446    n/a
4362    46992550    n/a
4363    67634569    n/a
4364    40402635    n/a

[4365 rows x 2 columns]

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [8]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Care-NA {attribute}-na - match upload_Shawn.csv',index=False)

In [9]:
looks_good_na('Bed Bath & Beyond', attribute, df, df_na)